In [10]:
seq_len = 24 * 4 * 4

border1s = [0, 12 * 30 * 24 + 4 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]

flag = 'val'
type_map = {'train': 0, 'val': 1, 'test': 2}
set_type = type_map[flag]

border1 = border1s[set_type]
border2 = border2s[set_type]

In [16]:
import torch

def calcute_lags(x_enc):
    q_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
    k_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
    res = q_fft * torch.conj(k_fft)
    corr = torch.fft.irfft(res, dim=-1)
    mean_value = torch.mean(corr, dim=1)
    _, lags = torch.topk(mean_value, 1, dim=-1)
    return lags

In [21]:
x_enc = torch.tensor([[
[1,2,3],
[4,5,6]
],
[
[7,8,9],
[10,11,12]
]
])
B, T, N = x_enc.size()
print(B, T, N)
x_enc = x_enc.permute(0, 2, 1).contiguous().reshape(B * N, T, 1)
min_values = torch.min(x_enc, dim=1)[0]
max_values = torch.max(x_enc, dim=1)[0]
medians = torch.median(x_enc, dim=1).values
lags = calcute_lags(x_enc)
trends = x_enc.diff(dim=1).sum(dim=1)

2 2 3


In [23]:
print(x_enc.shape)

torch.Size([6, 2, 1])
